### Install Dependencies

In [96]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Import Wikipedia Page

In [97]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

### Create Dataframe

In [98]:
tbody = soup.find('table').tbody
array = []
for tr in tbody.find_all('tr'):
    sub_array = []
    for th in tr.find_all('th'):
        sub_array.append(th.text.replace('\n', ''))
    for td in tr.find_all('td'):
        sub_array.append(td.text.replace('\n', ''))
    array.append(sub_array)
np_array = np.array(array)
df = pd.DataFrame(np_array)
df.columns = df.iloc[0]
df = df[1:]
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Remove rows where Borough is not assigned

In [99]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


### Combine rows of similiar Postcode

In [100]:
df = df.groupby('Postcode').agg({'Borough':'first', 
                             'Neighbourhood': ', '.join}).reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Change values of Neighbourhood to Borough where Neighbourhood is Not Assigned

In [101]:
df['Neighbourhood'] = df.apply(
    lambda row: row['Borough'] if row['Neighbourhood'] == 'Not assigned' else row['Neighbourhood'],
    axis=1
)

### Shape

In [102]:
df.shape

(103, 3)